# USU Wom*n's Trial Debate Allocation Algorithm
Developed by Viran Weerasekera (2017-18)

In [10]:
import gurobipy as grb
import pandas as pd
import numpy as np
import math
import datetime

def allocate(swing):
    global times
    alltimes = {1:"5:15pm",2:"6:15pm",3:"7:15pm",4:"8:15pm",5:"9:15pm",6:"10:15pm",7:"11:15pm",8:"12:15am"}
    
    # Import data
    global availabilities
    availabilities = {}
    global vetoes
    vetoes = {}
    global speakers
    speakers = []
    
    csv = pd.read_csv("2018 Wom*n's Trials (Responses) - Form responses 1.csv",index_col=0)

    for index,row in csv.iterrows():
        speakers.append(row[0])
        
        #No speaker position or availability vetoes in Wom*n's
        #vetoes[row[0]] = row[13]
        #availabilities[row[0]] = {}
        #availabilities[row[0]][1] = row[14]
        #availabilities[row[0]][2] = row[15]
        #availabilities[row[0]][3] = row[16]
        #availabilities[row[0]][4] = row[17]
        #availabilities[row[0]][5] = row[18]
        #availabilities[row[0]][6] = row[19]
        #availabilities[row[0]][7] = row[20]
        #availabilities[row[0]][8] = row[21]
        
    #Form Feasible Team Set (FTS)
    print("{} speakers to allocate{}".format(len(speakers),", swings added" if len(speakers) % 8 >=0 and swing else ""))
    print("-----")
    global fts
    fts = {}
    index = 0
    for speaker1 in speakers:
        for speaker2 in speakers:
            if speaker1 != speaker2:
                    fts[index] = [speaker1,speaker2]
                    index += 1
    if swing:
        #Swing Construction
        if len(speakers) % 8 >= 2:
            fts[index] = ["Imogen Harper 1","Imogen Harper 2"]
            index += 1
            speakers.append("Imogen Harper 1")
            speakers.append("Imogen Harper 2")
        if len(speakers) % 8 >= 4:
            fts[index] = ["Swing C","Swing D"]
            index += 1
            speakers.append("Swing C")
            speakers.append("Swing D")
        if len(speakers) % 8 >= 6:
            fts[index] = ["Swing E","Swing F"]
            index += 1
            speakers.append("Swing E")
            speakers.append("Swing F")
            
    #Calculate how many timeslots are required
    alltimeslots = (1,2,3,4,5,6,7,8)
    debatesreqd = int(np.floor(len(speakers)/8))
    timeslots = alltimeslots[:debatesreqd]
    times = {}
    
    #Generate Costs
    costs = {}
    for key,time in alltimes.iteritems():
        if key in timeslots:
            times[key] = time

    for key,team in fts.iteritems():
        for slot in timeslots:
            costs[key,slot] = 0
                    
        for speaker in team:
            rlvconflicts = []
            for conflictobject in conflicts:
                if speaker in conflictobject:
                    rlvconflicts.append(conflictobject)
        for rlvconflict in rlvconflicts:
            for spk in rlvconflict:
                if spk in team:
                    for slot in timeslots:
                        costs[key,slot] = grb.GRB.INFINITY
                    
    #Optimisation
    a = grb.Model("2018 Wom*n's Trial Allocator")
    x = {}
    for key,value in costs.iteritems():
        x[key] = a.addVar(vtype = grb.GRB.INTEGER, obj = value, name = "x_{}".format(key))

    #Exactly 4 teams in each timeslot
    for slot in timeslots:
        a.addConstr(sum(x[team,slot] for team in fts.keys()) == 4)

    #Each team to no more than 1 timeslot
    for key,team in fts.iteritems():
        a.addConstr(sum(x[key,slot] for slot in timeslots) <= 1)

    #Each speaker to exactly 1 timeslot
    #NOTE: Will cause crash if there are an odd number of speakers
    #TODO: Early exit for odd number
    for speaker in speakers:
        affectedTeams = []
        for key,team in fts.iteritems():
            if speaker in team:
                affectedTeams.append(key)
        a.addConstr(sum(x[team,slot] for team in affectedTeams for slot in timeslots) <= 1)
        
    a.modelSense = grb.GRB.MINIMIZE
    a.optimize()
  
    allocatedSpeakers = []
    unallocatedSpeakers = []
    print("-----")
    print("USU WOM*N'S TRIALS 2018 - DEBATE ALLOCATION")
    #print("Speakers in each team are listed in random order.")
    #print("Teams are listed in essentially random order.")
    print("***Number of triallists is not a multiple of 4 - {} speakers will not be allocated.".format(len(speakers) % 8) if len(speakers) % 8 > 0 else "Allocation generated at {}".format(datetime.datetime.now()))
    print("-----")
    for key,value in costs.iteritems():
        if x[key].x != 0:
            print("Team {} {} at {}".format(key[0],fts[key[0]],times[key[1]]))
            for speaker in fts[key[0]]:
                allocatedSpeakers.append(speaker)

    for speaker in speakers:
        if speaker not in allocatedSpeakers:
            unallocatedSpeakers.append(speaker)
    print("-----")
    print("Successfully allocated {} out of {} speakers across {} debates").format(len(allocatedSpeakers),len(speakers),debatesreqd)
    print("Unallocated Speakers: {}".format(unallocatedSpeakers) if len(unallocatedSpeakers) > 0 else "No unallocated speakers")
    print("-----")
    a.write("2018 Wom*n's Allocation.rlp")
    print("-----" if len(unallocatedSpeakers)>0 else "")
    print("Unallocated Speaker Information:" if len(unallocatedSpeakers)>0 else "")
    for s in unallocatedSpeakers:
        report(s)

conflicts = []
def addConflict(conflicter,conflictee):
    conflicts.append([conflicter,conflictee])
    
def report(speaker):
    rlv = []
    for conflict in conflicts:
        if conflict[0] == speaker:
            rlv.append(conflict[1])
        elif conflict[1] == speaker:
            rlv.append(conflict[0])
    if len(rlv) == 0:
        print("{} has no conflicts.".format(speaker))
    else:
        print("{} cannot debate with: {}".format(speaker, rlv))